### Notes: If ImportError occurs, it's probably due to the huggingface-hub. 
> pip install huggingface-hub==0.25.0
> 
> Reference: https://replicate.com/blog/how-to-prompt-llama

## Import

In [28]:
import torch
from transformers import pipeline
import re
import os

from langchain import HuggingFacePipeline, PromptTemplate
from transformers import pipeline
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from transformers import AutoTokenizer, TextStreamer, pipeline, BitsAndBytesConfig, AutoModelForCausalLM
import os
from tqdm import tqdm
import pandas as pd
import itertools, csv


DEVICE = "cuda:0" if torch.cuda.is_available() else "CPU"
HUGGING_FACE_TOKEN = os.environ.get('HUGGING_FACE_TOKEN') #in terminal: export HUGGING_FACE_TOKEN="YOUR_TOKEN"


## Load Model

In [23]:
'''
Possible Models:
- meta-llama/Llama-3.2-1B-Instruct
- meta-llama/Llama-3.2-3B-Instruct
- meta-llama/Llama-3.2-11B-Vision-Instruct
- meta-llama/Llama-3.1-70B-Instruct
'''
model_id = "meta-llama/Llama-3.2-3B-Instruct" 

# Quanitisize your model dtype (for sparsity)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

# Set token using ENV variable
tokenizer = AutoTokenizer.from_pretrained(model_id, token=HUGGING_FACE_TOKEN)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    token=HUGGING_FACE_TOKEN,
    quantization_config=bnb_config
)

augmentation_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# Data

In [66]:
directory_path = "/home/dongkyu/exported_docs"

In [67]:
import os
import re

def preprocess_text(text):
    """
    Cleans and preprocesses text for LLaMA dataset.

    Args:
        text (str): The raw text to preprocess.

    Returns:
        str: Cleaned and normalized text.
    """
    # Normalize whitespace and remove excessive newlines
    text = re.sub(r'\s+', ' ', text)  # Replace all whitespace (including newlines) with a single space
    text = re.sub(r'\n+', '\n', text.strip())  # Replace multiple newlines with a single newline
    # Remove any unwanted characters or patterns (URLs, etc.)
    text = re.sub(r'http[s]?://\S+', '', text)  # Remove URLs
    return text.strip()

def load_txt_files_to_text_format(directory, output_file):
    """
    Converts `.txt` files from a nested directory structure into a single `.txt` file with preprocessing.

    Args:
        directory (str): Root directory containing `.txt` files (including subdirectories).
        output_file (str): Path to save the combined and cleaned text file.

    Returns:
        None
    """
    with open(output_file, 'w', encoding='utf-8') as output_f:
        # Recursively traverse the directory
        for root, _, files in os.walk(directory):
            for file_name in sorted(files):
                if file_name.endswith('.txt'):  # Only process `.txt` files
                    file_path = os.path.join(root, file_name)
                    with open(file_path, 'r', encoding='utf-8') as f:
                        content = f.read()
                        cleaned_text = preprocess_text(content)
                        # Write the cleaned text to the output file
                        output_f.write(cleaned_text + '\n')  # Add a newline between entries

    print(f"Processed `.txt` files from nested directories saved to {output_file}.")

# Example usage
directory = directory_path  # Replace with your root directory with subdirectories
output_file = "finetune.txt"  # Output file path
load_txt_files_to_text_format(directory, output_file)


Processed `.txt` files from nested directories saved to finetune.txt.


In [68]:
def read_text_file_lines(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.readlines()
    except FileNotFoundError:
        print(f"Error: The file at {file_path} does not exist.")
    except Exception as e:
        print(f"An error occurred: {e}")

file_path = "finetune.txt"
lines = read_text_file_lines(file_path)


In [69]:
lines[0]

'Title: (SLAM) Navigating While Mapping\uf0c1 URL:  Section: getting_started/index.html -------------------------------------------------------------------------------- ## Overview\uf0c1 This document explains how to use Nav2 with SLAM. The following steps show ROS 2 users how to generate occupancy grid maps and use Nav2 to move their robot around. This tutorial applies to both simulated and physical robots, but will be completed here on a physical robot. Before completing this tutorial, completing theGetting Startedis highly recommended especially if you are new to ROS and Navigation2. In this tutorial we’ll be using SLAM Toolbox. More information can be found in theROSCon talk for SLAM Toolbox ## Requirements\uf0c1 You must install Navigation2, Turtlebot3, and SLAM Toolbox. If you don’t have them installed, please followGetting Started. SLAM Toolbox can be installed via: or from built from source in your workspace with: ## Tutorial Steps\uf0c1 ## 0- Launch Robot Interfaces\uf0c1 For 

# Generate
- '<|pad|>'
- '<|begin_of_text|>'
- '<|eot_id|>'

In [70]:
def write_instruction(note, pipe, max_new_tokens=100, num_beams=1, temperature=1.0):
    messages = [
    {
        "role": "system",
        "content": ("You are a helpful assistant. Your task is to make a fine-tuning dataset. Given a ANSWER, infer the QUERY. Think carefully and write the appropriate QUERY that would derive this ANSWER. Write only the query. Example: [ANSWER] 4 [/ANSWER] [QUERY] What is 2+2? [/QUERY]"),
    },
    {
        "role": "user",
        "content": (
            "[ANSWER]: {noteA} [/ANSWER]".format(noteA= note)
            ),
    }
]

    outputs = pipe(messages, max_new_tokens=max_new_tokens)
    answer = outputs[0]["generated_text"][-1]
    return answer


In [71]:
def generate_fine_tuning_dataset(lines, pipe, output_csv_path, max_new_tokens=100, num_beams=1, temperature=1.0):
    """
    Generates a fine-tuning dataset with queries and input lines, then saves it to a CSV file.
    
    Args:
        lines (list): A list of text strings (answers).
        pipe: The pipeline function for generating queries.
        output_csv_path (str): Path to save the resulting CSV file.
        max_new_tokens (int): Maximum number of new tokens to generate.
        num_beams (int): Beam search parameter for the model.
        temperature (float): Sampling temperature for the model.
    """
    results = []

    for line in lines:
        query = write_instruction(
                note=line,
                pipe=pipe,
                max_new_tokens=max_new_tokens,
                num_beams=num_beams,
                temperature=temperature
            )

        query= query['content']
        print(query)
        results.append({"query": query, "input": line})
    # Save results to CSV
    with open(output_csv_path, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=["query", "input"])
        writer.writeheader()
        writer.writerows(results)

    print(f"Dataset saved to {output_csv_path}")

# Example usage
output_csv_path = "fine_tuning_dataset.csv"
generate_fine_tuning_dataset(lines, augmentation_pipeline, output_csv_path)


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[QUERY] What is the best way to enable SLAM on a turtlebot3? [/QUERY]


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[QUERY] How to enable the STVL plugin in Navigation2? [/QUERY]


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

[QUERY] What is the code for the Fibonacci action server? [/QUERY]


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[QUERY] What is the recommended way to register resources for a ROS package? [/QUERY]


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


What is the command to install eProsima Fast DDS?
[QUERY] How to build ROS 2 on macOS? [/QUERY]
Dataset saved to fine_tuning_dataset.csv


# Make into ShareGPT Format

In [72]:
df= pd.read_csv('fine_tuning_dataset.csv')

In [73]:
def clean_special_tokens(df):
    """
    Removes special tokens [QUERY], [/QUERY], [ANSWER], and [/ANSWER] from the 'query' and 'input' columns in a DataFrame.
    
    Args:
        df (pd.DataFrame): The input DataFrame with 'query' and 'input' columns.

    Returns:
        pd.DataFrame: A new DataFrame with cleaned columns.
    """
    # Define the list of special tokens to remove
    special_tokens = ["[QUERY]", "[/QUERY]", "[ANSWER]", "[/ANSWER]", "[TEXT]", "[/TEXT]" ]
    
    # Remove special tokens from both columns
    for token in special_tokens:
        df['query'] = df['query'].str.replace(token, "", regex=False)
        df['input'] = df['input'].str.replace(token, "", regex=False)
    
    # Optionally strip leading and trailing whitespace
    df['query'] = df['query'].str.strip()
    df['input'] = df['input'].str.strip()
    
    return df

In [74]:
cleaned_df = clean_special_tokens(df)

In [75]:
cleaned_df

,query,input
0,What is the best way to enable SLAM on a turtl...,Title: (SLAM) Navigating While Mapping URL: ...
1,How to enable the STVL plugin in Navigation2?,Title: (STVL) Using an External Costmap Plugin...
2,What are the parameters for the AMCL navigatio...,Title: AMCL URL: Section: getting_started/in...
3,What is the purpose of the Navigation Stack in...,Title: About and Contact URL: Section: getti...
4,What is the primary state of a Lifecycle node ...,Title: Adding a New Nav2 Task Server URL: Se...
...,...,...
440,What is the purpose of the `visibility_control...,Title: Writing an action server and client (C+...
441,What is the code for the Fibonacci action server?,Title: Writing an action server and client (Py...
442,What is the recommended way to register resour...,Title: ament_cmake user documentation URL: S...
443,What is the command to install eProsima Fast DDS?,Title: eProsima Fast DDS URL: Section: Insta...


In [87]:
import pandas as pd
import json


def dataframe_to_jsonl_with_roles(df, output_path):
    """
    Converts a DataFrame with 'query' and 'input' columns into a JSONL format,
    where each JSON element contains 'system', 'user', and 'assistant' roles.

    Args:
        df (pd.DataFrame): Input DataFrame with 'query' and 'input' columns.
        output_path (str): Path to save the JSONL file.

    Returns:
        None
    """
    with open(output_path, 'w') as f:
        for _, row in df.iterrows():
            conversation = {
                "system": {"role": "system", "content": "You are an assistant"},
                "user": {"role": "user", "content": row['query']},
                "assistant": {"role": "assistant", "content": row['input']}
            }
            # Write each conversation as a single JSON object
            f.write(json.dumps(conversation) + '\n')




# Convert the DataFrame to JSON format and save it
output_file = "finetune.jsonl"
dataframe_to_jsonl_with_roles(cleaned_df, output_file)

print(f"JSON saved to {output_file}")


JSON saved to finetune.jsonl
